In [ ]:
import json
import pickle
import time
import warnings
from pathlib import Path

import catboost
import lightgbm as lgb
import mapie
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import requests
import xgboost
from IPython.display import clear_output, display
from pymongo import MongoClient
from pymongoarrow.api import find_pandas_all
from sklearn import (compose, dummy, ensemble, impute, linear_model, metrics,
                     model_selection, pipeline, preprocessing, svm, tree)
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm import tqdm

import creds

# Get data from MongoDB

In [ ]:
cluster = MongoClient(creds.Creds.URI)

query = {"building_condition": "Good"}
df = find_pandas_all(cluster.development.BE_houses, None)
df

# Show choropleth data

In [ ]:
BE_provinces = requests.get(
    "https://raw.githubusercontent.com/mathiasleroy/Belgium-Geographic-Data/master/dist/polygons/be-provinces-unk-WGS84.geo.json"
).json()

aggregate = (
    df.assign(list_price=lambda df: pd.to_numeric(df.list_price))
    .groupby("province")
    .agg(
        list_price_count=("list_price", "count"),
        list_price_mean=("list_price", "median"),
    )
    .reset_index()
)

fig = px.choropleth(
    aggregate,
    geojson=BE_provinces,
    locations="province",
    color="list_price_mean",
    featureidkey="properties.name",
    projection="mercator",
    color_continuous_scale="Magenta",
    labels={
        "list_price_mean": "Median Price",
        "list_price_count": "Number of Observations",
    },
    hover_data={"list_price_mean": ":.3s", "province": True, "list_price_count": True},
)

fig.update_geos(
    showcountries=True, showcoastlines=True, showland=True, fitbounds="locations"
)

# Add title and labels
fig.update_layout(
    title_text="Median House Prices by Province",
    autosize=False,
    width=800,
    height=600,
    geo=dict(showframe=False, showcoastlines=False, projection_type="mercator"),
)


fig.show()

# Function for preparing data after loading from MongoDB

In [ ]:
def prepare_data_for_modeling(df):
    columns_to_keep = [
        "construction_year",
        "building_condition",
        "number_of_frontages",
        "living_area",
        "bedrooms",
        "bathrooms",
        "toilets",
        "surface_of_the_plot",
        "primary_energy_consumption",
        "energy_class",
        "heating_type",
        "tenement_building",
        "list_price",
        "double_glazing",
        "province",
    ]

    return (
        df.filter(columns_to_keep)
        .assign(
            construction_year=lambda df: pd.to_numeric(
                df.construction_year, downcast="unsigned", errors="coerce"
            ),
            number_of_frontages=lambda df: pd.to_numeric(
                df.number_of_frontages, downcast="unsigned", errors="coerce"
            ),
            living_area=lambda df: pd.to_numeric(
                df.living_area, downcast="unsigned", errors="coerce"
            ),
            bedrooms=lambda df: pd.to_numeric(
                df.bedrooms, downcast="unsigned", errors="coerce"
            ),
            bathrooms=lambda df: pd.to_numeric(
                df.bathrooms, downcast="unsigned", errors="coerce"
            ),
            toilets=lambda df: pd.to_numeric(
                df.toilets, downcast="unsigned", errors="coerce"
            ),
            surface_of_the_plot=lambda df: pd.to_numeric(
                df.surface_of_the_plot, downcast="unsigned", errors="coerce"
            ),
            primary_energy_consumption=lambda df: pd.to_numeric(
                df.primary_energy_consumption, downcast="unsigned", errors="coerce"
            ),
            list_price=lambda df: np.log10(
                pd.to_numeric(df.list_price, downcast="unsigned", errors="coerce")
            ),
            building_condition=lambda df: df.building_condition.fillna(
                "Unknown"
            ).astype("category"),
            energy_class=lambda df: df.energy_class.fillna("Unknown").astype(
                "category"
            ),
            heating_type=lambda df: df.heating_type.fillna("Unknown").astype(
                "category"
            ),
            tenement_building=lambda df: df.tenement_building.fillna("Unknown").astype(
                "category"
            ),
            double_glazing=lambda df: df.double_glazing.fillna("Unknown").astype(
                "category"
            ),
            province=lambda df: df.province.fillna("Unknown").astype("category"),
        )
        .dropna(subset=["list_price"])
    )


processed_df = prepare_data_for_modeling(df)
processed_df

# Obtaining conformity intervals


In [ ]:
y = processed_df["list_price"]
X = processed_df.drop(
    columns=[
        "list_price",
        "building_condition",
        "energy_class",
        "heating_type",
        "tenement_building",
        "double_glazing",
        "province",
    ]
)

In [ ]:
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(
    X, y, test_size=0.2
)

In [ ]:
catboost_train = catboost.Pool(
    X_train,
    y_train,
    # cat_features=X.select_dtypes(include='category').columns.tolist(),
)

catboost_valid = catboost.Pool(
    X_valid,
    y_valid,
    # cat_features=X.select_dtypes(include='category').columns.tolist(),
)

model = catboost.CatBoostRegressor(
    loss_function="RMSE",
)
model.fit(
    catboost_train,
    eval_set=[catboost_valid],
    early_stopping_rounds=100,
    verbose=False,
    use_best_model=True,
)

mapie_model = mapie.regression.MapieRegressor(model, method="plus")

# fit MAPIE model
mapie_model.fit(X_train, y_train)

# make predictions with prediction intervals
y_pred, y_pis = mapie_model.predict(X_valid, alpha=0.1)

In [ ]:
# Create a DataFrame with y_valid and prediction intervals
conformal_df = pd.DataFrame(
    {
        "y_valid": y_valid,
        "lower": y_pis[:, 0].flatten(),
        "upper": y_pis[:, 1].flatten(),
        "y_pred": y_pred,
    }
)

# Sort the DataFrame by y_valid
df_sorted = conformal_df.sort_values(by="y_valid")

# Plot prediction intervals
plt.fill_between(
    df_sorted["y_valid"],
    df_sorted["lower"],
    df_sorted["upper"],
    alpha=0.5,
    color="gray",
    label="Prediction Intervals",
)
plt.scatter(
    df_sorted["y_valid"], df_sorted["y_pred"], color="blue", label="Predictions"
)

plt.legend()
plt.show()

In [ ]:
10**df

# Comparing different models

In [ ]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    """
    A transformer for selecting specific columns from a DataFrame.

    This class inherits from the BaseEstimator and TransformerMixin classes from sklearn.base.
    It overrides the fit and transform methods from the parent classes.

    Attributes:
        feature_names_in_ (list): The names of the features to select.
        n_features_in_ (int): The number of features to select.

    Methods:
        fit(X, y=None): Fit the transformer. Returns self.
        transform(X, y=None): Apply the transformation. Returns a DataFrame with selected features.
    """

    def __init__(self, feature_names_in_):
        """
        Constructs all the necessary attributes for the FeatureSelector object.

        Args:
            feature_names_in_ (list): The names of the features to select.
        """
        self.feature_names_in_ = feature_names_in_
        self.n_features_in_ = len(feature_names_in_)

    def fit(self, X, y=None):
        """
        Fit the transformer. This method doesn't do anything as no fitting is necessary.

        Args:
            X (DataFrame): The input data.
            y (array-like, optional): The target variable. Defaults to None.

        Returns:
            self: The instance itself.
        """
        return self

    def transform(self, X, y=None):
        """
        Apply the transformation. Selects the features from the input data.

        Args:
            X (DataFrame): The input data.
            y (array-like, optional): The target variable. Defaults to None.

        Returns:
            DataFrame: A DataFrame with only the selected features.
        """
        return X.loc[:, self.feature_names_in_].copy(deep=True)

In [ ]:
processed_df = prepare_data_for_modeling(df)
y = processed_df["list_price"]
X = processed_df.drop(columns=["list_price"])

X_train, X_valid, y_train, y_valid = model_selection.train_test_split(
    X, y, test_size=0.2
)

In [ ]:
# Selecting columns by dtypes

numerical_columns = X_train.head().select_dtypes("number").columns.to_list()
categorical_columns = X_train.head().select_dtypes("category").columns.to_list()

In [ ]:
# Prepare pipelines for corresponding columns:
numerical_pipeline = pipeline.Pipeline(
    steps=[
        ("num_selector", FeatureSelector(numerical_columns)),
        ("imputer", impute.SimpleImputer(strategy="median")),
        ("std_scaler", preprocessing.MinMaxScaler()),
    ]
)

categorical_pipeline = pipeline.Pipeline(
    steps=[
        ("cat_selector", FeatureSelector(categorical_columns)),
        ("imputer", impute.SimpleImputer(strategy="most_frequent")),
        (
            "onehot",
            preprocessing.OneHotEncoder(handle_unknown="ignore", sparse_output=True),
        ),
    ]
)

# Put all the pipelines inside a FeatureUnion:
data_preprocessing_pipeline = pipeline.FeatureUnion(
    n_jobs=-1,
    transformer_list=[
        ("numerical_pipeline", numerical_pipeline),
        ("categorical_pipeline", categorical_pipeline),
    ],
)

In [ ]:
[
    linear_model.LinearRegression(),
    linear_model.SGDRegressor(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.RANSACRegressor(),
    linear_model.Lasso(),
    svm.SVR(),
    ensemble.GradientBoostingRegressor(),
    tree.DecisionTreeRegressor(),
    ensemble.RandomForestRegressor(),
    ensemble.ExtraTreesRegressor(),
    ensemble.AdaBoostRegressor(),
    catboost.CatBoostRegressor(silent=True),
    lgb.LGBMRegressor(verbose=-1),
    xgboost.XGBRegressor(verbosity=0),
    dummy.DummyClassifier(),
]

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)

    MLA = [
        linear_model.LinearRegression(),
        linear_model.SGDRegressor(),
        linear_model.PassiveAggressiveRegressor(),
        linear_model.RANSACRegressor(),
        linear_model.Lasso(),
        svm.SVR(),
        ensemble.GradientBoostingRegressor(),
        tree.DecisionTreeRegressor(),
        ensemble.RandomForestRegressor(),
        ensemble.ExtraTreesRegressor(),
        ensemble.AdaBoostRegressor(),
        catboost.CatBoostRegressor(silent=True),
        lgb.LGBMRegressor(verbose=-1),
        xgboost.XGBRegressor(verbosity=0),
        dummy.DummyClassifier(),
    ]

    # note: this is an alternative to train_test_split
    cv_split = model_selection.ShuffleSplit(
        n_splits=10, test_size=0.3, train_size=0.6, random_state=0
    )  # run model 10x with 60/30 split intentionally leaving out 10%

    # create table to compare MLA metrics
    MLA_columns = [
        "MLA Name",
        "MLA Parameters",
        "MLA Train RMSE Mean",
        "MLA Test RMSE Mean",
        "MLA Train R2 Mean",
        "MLA Test R2 Mean",
        "MLA Time",
    ]
    MLA_compare = pd.DataFrame(columns=MLA_columns)

    # index through MLA and save performance to table
    row_index = 0
    for alg in tqdm(MLA):
        # set name and parameters
        MLA_name = alg.__class__.__name__
        MLA_compare.loc[row_index, "MLA Name"] = MLA_name
        MLA_compare.loc[row_index, "MLA Parameters"] = str(alg.get_params())

        model_pipeline = pipeline.Pipeline(
            steps=[
                ("data_preprocessing_pipeline", data_preprocessing_pipeline),
                ("model", alg),
            ]
        )

        cv_results = model_selection.cross_validate(
            model_pipeline,
            X_train,
            y_train,
            cv=cv_split,
            scoring={
                "r2": "r2",
                "neg_root_mean_squared_error": "neg_root_mean_squared_error",
            },
            return_train_score=True,
        )

        MLA_compare.loc[row_index, "MLA Time"] = cv_results["fit_time"].mean()
        MLA_compare.loc[row_index, "MLA Train RMSE Mean"] = cv_results[
            "train_neg_root_mean_squared_error"
        ].mean()
        MLA_compare.loc[row_index, "MLA Test RMSE Mean"] = cv_results[
            "test_neg_root_mean_squared_error"
        ].mean()

        MLA_compare.loc[row_index, "MLA Train R2 Mean"] = cv_results["train_r2"].mean()
        MLA_compare.loc[row_index, "MLA Test R2 Mean"] = cv_results["test_r2"].mean()

        row_index += 1

        clear_output(wait=True)
        display(MLA_compare.sort_values(by=["MLA Test RMSE Mean"], ascending=False))

In [ ]:
with open("scraped_data.pickle", "rb") as handle:
    data = pickle.load(handle)

In [ ]:
def convert_zip_to_province(value):
    # data from https://www.spotzi.com/en/data-catalog/categories/postal-codes/belgium/
    if value is None:
        return None

    first_two_digits = int(value[:2])

    province_dict = {
        range(10, 13): "Brussels",
        range(13, 15): "Walloon Brabant",
        range(15, 20): "Flemish Brabant",
        range(30, 35): "Flemish Brabant",
        range(20, 30): "Antwerp",
        range(35, 40): "Limburg",
        range(40, 50): "Liege",
        range(50, 60): "Namur",
        range(60, 66): "Hainaut",
        range(70, 80): "Hainaut",
        range(66, 70): "Luxembourg",
        range(80, 90): "West Flanders",
        range(90, 100): "East Flanders",
    }

    for key in province_dict:
        if first_two_digits in key:
            return province_dict[key]

    return None


def reformat_entries(data):
    return (
        data.apply(lambda x: x.astype(str))
        .assign(
            price=lambda x: pd.to_numeric(
                x["price"].str.extract(r"(\d+,\d+)", expand=False).str.replace(",", ""),
                errors="coerce",
            ),
            province=lambda x: x["zip_code"].apply(convert_zip_to_province),
            energy_class=lambda x: x["energy_class"].str.strip(),
            primary_energy_consumption=lambda x: pd.to_numeric(
                x["primary_energy_consumption"]
                .str.extract(r"(\d+)", expand=False)
                .str.replace(",", ""),
                errors="coerce",
            ),
            bedrooms=lambda x: pd.to_numeric(
                x["bedrooms"].str.extract(r"(\d+)", expand=False), errors="coerce"
            ),
            tenement_building=lambda x: x["tenement_building"].str.strip(),
            living_area=lambda x: pd.to_numeric(
                x["living_area"]
                .str.extract(r"(\d+)", expand=False)
                .str.replace(",", ""),
                errors="coerce",
            ),
            surface_of_the_plot=lambda x: pd.to_numeric(
                x["surface_of_the_plot"]
                .str.extract(r"(\d+)", expand=False)
                .str.replace(",", ""),
                errors="coerce",
            ),
            bathrooms=lambda x: pd.to_numeric(
                x["bathrooms"].str.extract(r"(\d+)", expand=False), errors="coerce"
            ),
            double_glazing=lambda x: x["double_glazing"].str.strip(),
            number_of_frontages=lambda x: pd.to_numeric(
                x["number_of_frontages"].str.extract(r"(\d+)", expand=False),
                errors="coerce",
            ),
            building_condition=lambda x: x["building_condition"].str.strip(),
            toilets=lambda x: pd.to_numeric(
                x["toilets"].str.extract(r"(\d+)", expand=False), errors="coerce"
            ),
            heating_type=lambda x: x["heating_type"].str.strip(),
            construction_year=lambda x: pd.to_numeric(
                x["construction_year"].str.extract(r"(\d+)", expand=False),
                errors="coerce",
            ),
        )
        .transpose()
        .squeeze()
        .to_dict()
    )

In [ ]:
testy = data[1].apply(lambda x: x.astype(str))

In [ ]:
reformat_entries(data[0].apply(lambda x: x.astype(str)))

In [ ]:
for i in range(10):
    print(data[i].construction_year)